# RSA with unknown n and weak prime generation

$c = pow(m,2,n)$

In [132]:
from Crypto.Util.number import *
from sympy import *
from pwn import *

In [138]:
bytes.fromhex("0FFF")

old = bytes.fromhex
bytes.fromhex = lambda x: old("0"+x)if len(x)%2 == 1 

b'\x0f\xff'

In [133]:
r = remote("rabin.challenge.cryptonite.team",1337)
r.recvuntil(b">>")
r.send(b"G\n")
r.recvuntil(b"Your encrypted flag is:\n")
flag = bytes_to_long(bytes.fromhex(r.recvuntil(b"\n")[:-1].decode("ASCII")))
r.recvuntil(b">>")

print(flag)

def oracle(payload):
    r.send(b"E\n")
    r.recvuntil(b"Enter message in hex:\n")
    to_hex = long_to_bytes(payload).hex()
    r.send(bytes(to_hex,"ASCII") + b"\n")
    r.recvuntil(b"is: \n")
    ciphertext = bytes_to_long(bytes.fromhex(r.recvuntil(b"\n")[:-1].decode("ASCII")))
    r.recvuntil(b">>")
    return ciphertext

oracle(0xFF),oracle(0xFF)


[x] Opening connection to rabin.challenge.cryptonite.team on port 1337
[x] Opening connection to rabin.challenge.cryptonite.team on port 1337: Trying 34.141.169.115
[+] Opening connection to rabin.challenge.cryptonite.team on port 1337: Done
884034210547077656651046986700408217763418130786925137494132992370619976462437407478594103692015302140841705313869227202035671190061331119436258746094089
b'Enter message in hex:\n'
b'ff\n'
b'Your Ciphertext is: \n'
b'Enter message in hex:\n'
b'ff\n'
b'Your Ciphertext is: \n'


(65025, 65025)

In [134]:
def solve(flag):

    # 2^512 bit mod, therefore will (2^256)^2 likely lead to mod reduction
    start = 2**255
    end = 2**256

    while start + 1 < end:
        midpoint = int(start + (end-start)//2)

        if oracle(midpoint) == midpoint**2:
            start = midpoint
        else:
            end = midpoint

    r = 1

    test_x = end+1
    test_y = oracle(test_x)

    # r should likely be one, but we just check to make sure
    while True:
        n, mod = divmod(test_x**2 - test_y,r)

        if mod == 0 and test_y == pow(test_x,2,n):
            break
        r += 2

    assert N==n

    # femat factorization technique, because the primes are next to each other
    # http://deweger.xs4all.nl/papers/%5b33%5ddW-SmlPrDif-AAECC%5b2002%5d.pdf
    # https://facthacks.cr.yp.to/fermat.html
    import gmpy2

    basepoint = gmpy2.isqrt(gmpy2.mpz(n))

    while not gmpy2.is_square(basepoint**2 - n):
        basepoint +=1

    b = gmpy2.isqrt(gmpy2.mpz(basepoint**2 - n))
    a = basepoint

    p,q = int(a - b),int(a+b)

    assert p*q == N

    mp = pow(flag,(p+1)//4,p)
    mq = pow(flag,(q+1)//4,q)

    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = egcd(b % a, a)
            return (g, x - (b // a) * y, y)

    c,y_p,y_q = egcd(p,q)

    assert p*y_p + q*y_q ==1

    r1 = (y_p * p * mq + y_q * q * mp) % n
    r2 = n-r1
    r3 = (y_p * p * mq - y_q * q * mp) % n
    r4 = n-r3

    return [r1,r2,r3,r4]

In [135]:
res = solve(flag)

b'Enter message in hex:\n'
b'c000000000000000000000000000000000000000000000000000000000000000\n'
b'Your Ciphertext is: \n'
b'Enter message in hex:\n'
b'a000000000000000000000000000000000000000000000000000000000000000\n'
b'Your Ciphertext is: \n'
b'Enter message in hex:\n'
b'b000000000000000000000000000000000000000000000000000000000000000\n'
b'Your Ciphertext is: \n'
b'Enter message in hex:\n'
b'b800000000000000000000000000000000000000000000000000000000000000\n'
b'Your Ciphertext is: \n'


ValueError: non-hexadecimal number found in fromhex() arg at position 127